In [41]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import time
from scipy.spatial.transform import Rotation


In [42]:
all_pts_2d = np.loadtxt("/home/mullin/WorkSpace/CourseProject/3 VAMR/Exercise 2 - PnP/data/detected_corners.txt")

In [43]:
image_idx = 1
pts_2d = np.reshape(
    np.loadtxt("/home/mullin/WorkSpace/CourseProject/3 VAMR/Exercise 2 - PnP/data/detected_corners.txt")[image_idx-1,:],
    (-1, 2) )

In [44]:
# 内参矩阵
K = np.loadtxt("/home/mullin/WorkSpace/CourseProject/3 VAMR/Exercise 2 - PnP/data/K.txt")

In [45]:
# 点的世界坐标系
p_W_corners = 0.01 * np.loadtxt("/home/mullin/WorkSpace/CourseProject/3 VAMR/Exercise 2 - PnP/data/p_W_corners.txt", delimiter = ",")

In [46]:
# 世界坐标与图像坐标下的点
p = pts_2d
P = p_W_corners

## 位姿估计函数

In [8]:
def estimatePoseDLT(p, P, K):
    
    num_corners = p.shape[0]
    
    pts_2d_aug = np.c_[p, np.ones((p.shape[0], 1))]
    p_cam = (np.mat(K).I * pts_2d_aug.T).T [:,0:2]
    p_W_mat_h = np.hstack((P,np.mat(np.ones((num_corners,1)))))
    

    Q = np.mat(np.zeros((num_corners*2,12)))
    for i in range (num_corners):
        Q[2*i, 0:4] = p_W_mat_h[i]
        Q[2*i, 8:12] = p_W_mat_h[i] * (-1) * p_cam[i,0]
        Q[2*i+1, 4:8] = p_W_mat_h[i]
        Q[2*i+1, 8:12] = p_W_mat_h[i] * (-1) * p_cam[i,1]
        
    U,sigma, VT = np.linalg.svd(Q,full_matrices=True)
    M = VT.T[:,-1].reshape(3,4)
    if (np.linalg.det(M[:, :3]) < 0):
        M *= -1    
    R = M[0:3,0:3]
    U_R, Sigma_R, VT_R = np.linalg.svd(R)
    R_ = U_R*VT_R
    alpha = np.linalg.norm(R_) / np.linalg.norm(R)
    M_ = np.c_[R_, alpha * M[:,3]]
    
    return M_

In [47]:
M = estimatePoseDLT(p, P, K)

## Test on Reproject

In [48]:
p

array([[186.5, 187.5],
       [264.5, 166.5],
       [218.5, 304.5],
       [285.5, 270.5],
       [292.5, 369.5],
       [356.5, 322.5],
       [406.5, 418.5],
       [461.5, 353.5],
       [388.5, 204.5],
       [495.5, 211.5],
       [396.5, 280.5],
       [496.5, 300.5]])

In [49]:
P

array([[ 0.    ,  0.08  ,  0.    ],
       [ 0.104 ,  0.08  ,  0.    ],
       [ 0.    ,  0.215 ,  0.    ],
       [ 0.104 ,  0.215 ,  0.    ],
       [ 0.    ,  0.266 , -0.1095],
       [ 0.104 ,  0.266 , -0.1095],
       [ 0.    ,  0.266 , -0.2445],
       [ 0.104 ,  0.266 , -0.2445],
       [ 0.194 ,  0.146 , -0.1095],
       [ 0.194 ,  0.146 , -0.2445],
       [ 0.194 ,  0.25  , -0.1095],
       [ 0.194 ,  0.25  , -0.2445]])

In [50]:
num_corners = p.shape[0]
p_W_mat_h = np.hstack((P,np.mat(np.ones((num_corners,1)))))

In [51]:
p_W_mat_h

matrix([[ 0.    ,  0.08  ,  0.    ,  1.    ],
        [ 0.104 ,  0.08  ,  0.    ,  1.    ],
        [ 0.    ,  0.215 ,  0.    ,  1.    ],
        [ 0.104 ,  0.215 ,  0.    ,  1.    ],
        [ 0.    ,  0.266 , -0.1095,  1.    ],
        [ 0.104 ,  0.266 , -0.1095,  1.    ],
        [ 0.    ,  0.266 , -0.2445,  1.    ],
        [ 0.104 ,  0.266 , -0.2445,  1.    ],
        [ 0.194 ,  0.146 , -0.1095,  1.    ],
        [ 0.194 ,  0.146 , -0.2445,  1.    ],
        [ 0.194 ,  0.25  , -0.1095,  1.    ],
        [ 0.194 ,  0.25  , -0.2445,  1.    ]])

In [52]:
pts_2d_ = (K * M * p_W_mat_h.T).T

In [53]:
pts_2d_

matrix([[ 80.15271064,  80.52709773,   0.43046317],
        [133.18287885,  83.56509917,   0.50372897],
        [104.16217464, 145.01842383,   0.47746727],
        [157.19234285, 148.05642527,   0.55073307],
        [124.90605085, 158.15150018,   0.42750402],
        [177.93621906, 161.18950161,   0.50076982],
        [139.29820268, 144.30590817,   0.34401318],
        [192.32837089, 147.34390961,   0.41727898],
        [202.48596501, 106.49291812,   0.52239149],
        [216.87811683,  92.64732612,   0.43890065],
        [220.98214468, 156.17512489,   0.55860206],
        [235.3742965 , 142.32953289,   0.47511122]])

In [54]:
pts_2d_[:,0:2]

matrix([[ 80.15271064,  80.52709773],
        [133.18287885,  83.56509917],
        [104.16217464, 145.01842383],
        [157.19234285, 148.05642527],
        [124.90605085, 158.15150018],
        [177.93621906, 161.18950161],
        [139.29820268, 144.30590817],
        [192.32837089, 147.34390961],
        [202.48596501, 106.49291812],
        [216.87811683,  92.64732612],
        [220.98214468, 156.17512489],
        [235.3742965 , 142.32953289]])

In [55]:
pts_2d_[:,2]

matrix([[0.43046317],
        [0.50372897],
        [0.47746727],
        [0.55073307],
        [0.42750402],
        [0.50076982],
        [0.34401318],
        [0.41727898],
        [0.52239149],
        [0.43890065],
        [0.55860206],
        [0.47511122]])

In [56]:
pts_repro = pts_2d_[:,0:2] / pts_2d_[:,2]

In [57]:
pts_repro

matrix([[186.20108862, 187.07081945],
        [264.39392517, 165.89297939],
        [218.15563242, 303.72432288],
        [285.42383119, 268.8351822 ],
        [292.17514613, 369.94154693],
        [355.32536289, 321.88341675],
        [404.92112313, 419.47784889],
        [460.9107611 , 353.10647726],
        [387.6134428 , 203.8565321 ],
        [494.13943311, 211.08951828],
        [395.59851297, 279.58207781],
        [495.40884118, 299.57098121]])

### reprojectPoints

In [71]:
def reprojectPoints(P, M_tilde, K):
    # Reproject 3D points given a projection matrix
    #
    # P         [n x 3] coordinates of the 3d points in the world frame
    # M_tilde   [3 x 4] projection matrix
    # K         [3 x 3] camera matrix
    #
    # Returns [n x 2] coordinates of the reprojected 2d points
    

    # YOUR CODE GOES HERE
    num_corners = P.shape[0]
    p_W_mat_h = np.hstack((P,np.mat(np.ones((num_corners,1)))))
    pts_2d_ = (K * M_tilde * p_W_mat_h.T).T
    pts_repro = pts_2d_[:,0:2] / pts_2d_[:,2]
    return pts_repro

In [72]:
p_re = reprojectPoints(P, M, K)

In [73]:
p_re

matrix([[186.20108862, 187.07081945],
        [264.39392517, 165.89297939],
        [218.15563242, 303.72432288],
        [285.42383119, 268.8351822 ],
        [292.17514613, 369.94154693],
        [355.32536289, 321.88341675],
        [404.92112313, 419.47784889],
        [460.9107611 , 353.10647726],
        [387.6134428 , 203.8565321 ],
        [494.13943311, 211.08951828],
        [395.59851297, 279.58207781],
        [495.40884118, 299.57098121]])

In [74]:
p_re[:, 0]

matrix([[186.20108862],
        [264.39392517],
        [218.15563242],
        [285.42383119],
        [292.17514613],
        [355.32536289],
        [404.92112313],
        [460.9107611 ],
        [387.6134428 ],
        [494.13943311],
        [395.59851297],
        [495.40884118]])

In [76]:
p_re[:, 1]

matrix([[187.07081945],
        [165.89297939],
        [303.72432288],
        [268.8351822 ],
        [369.94154693],
        [321.88341675],
        [419.47784889],
        [353.10647726],
        [203.8565321 ],
        [211.08951828],
        [279.58207781],
        [299.57098121]])

## Answer

In [77]:
def reprojectPoints_solution(P, M_tilde, K):
    # Reproject 3D points given a projection matrix
    #
    # P         [n x 3] coordinates of the 3d points in the world frame
    # M_tilde   [3 x 4] projection matrix
    # K         [3 x 3] camera matrix
    #
    # Returns [n x 2] coordinates of the reprojected 2d points

    p_homo = (K @ M_tilde @ np.r_[P.T, np.ones((1, P.shape[0]))]).T
    return p_homo[:,:2]/p_homo[:,2,np.newaxis]

In [78]:
p_re = reprojectPoints_solution(P, M, K)

In [79]:
p_re

matrix([[186.20108862, 187.07081945],
        [264.39392517, 165.89297939],
        [218.15563242, 303.72432288],
        [285.42383119, 268.8351822 ],
        [292.17514613, 369.94154693],
        [355.32536289, 321.88341675],
        [404.92112313, 419.47784889],
        [460.9107611 , 353.10647726],
        [387.6134428 , 203.8565321 ],
        [494.13943311, 211.08951828],
        [395.59851297, 279.58207781],
        [495.40884118, 299.57098121]])